In [1]:
import pandas as pd
import numpy as np
import pickle
import io
import os
import json
import distutils.dir_util
from collections import Counter
from tqdm import tqdm

In [2]:
# train data 준비
f_train = pd.read_json('data/train_cnt3.json' ,typ = 'frame', encoding='utf-8')
df_train = pd.DataFrame(f_train)
df_train.head(3)

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[팝],64292,첫 곡부터 중독,"[341760, 406529, 101063, 181321, 367817, 96395...",10,2016-06-12 17:43:48.000
1,[위로],32376,당신을 위로 해줄 뮤직 테라피 위로Song,"[598532, 285708, 681614, 549392, 532114, 34280...",22,2016-02-24 15:49:15.000
2,"[추억, 밤, 새벽, 회상]",54886,생각나는사람이있다면 들어보세요,"[271235, 124293, 385543, 290571, 157965, 44084...",52,2016-07-10 00:13:36.000


In [3]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37406 entries, 0 to 37405
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tags          37406 non-null  object
 1   id            37406 non-null  int64 
 2   plylst_title  37406 non-null  object
 3   songs         37406 non-null  object
 4   like_cnt      37406 non-null  int64 
 5   updt_date     37406 non-null  object
dtypes: int64(2), object(4)
memory usage: 1.7+ MB


In [4]:
# 플레이리스트 내 song id 리스트
train_song = df_train['songs']
# 플레이리스트 내 tag 리스트
train_tag = df_train['tags']
# 플레이리스트 내 song id 리스트 전체 나열
song_list = [song for plist in train_song for song in plist]
# 플레이리스트 내 tag 리스트 전체 나열
tag_list = [tag for plist in train_tag for tag in plist]

In [5]:
# 전체 나열 리스트 중 각 song의 개수
count_song = Counter(song_list)
# 전체 나열 리스트 중 각 tag의 개수
count_tag = Counter(tag_list)

In [6]:
# 전체 플레이리스트 내 song을 dictionary에 추가
mfl={}  # meaningful
for key, value in count_song.items():
    mfl[key]=value
song_len = len(mfl)

88146

In [7]:
# 전체 플레이리스트 내 10회 넘게 담긴 tag을 dictionary에 추가
for key, value in count_tag.items():
     mfl[key]=value
tag_len=len(mfl)-song_len
tag_len

3551

In [8]:
mfl_col = list(mfl.keys())
len(mfl_col) # 유의미한 song과 tag의 총 개수 (song_len+tag_len)

91697

In [33]:
with open('pickle/mfl_col.pkl', 'wb') as f :
    pickle.dump(mfl_col, f)

In [9]:
# 각 플레이리스트에 해당하는 곡, 태그 좌표에 1 부여

def create_onehot(df,column_name):
    zero_matrix=np.zeros((len(df),len(column_name)))
    zero_df=pd.DataFrame(zero_matrix,columns=column_name,index=df['id'])
    for i in tqdm(range(len(df))):
        for j in df.iloc[i,0]+df.iloc[i,3] :
            if j in column_name:
                zero_df.iloc[i,column_name.index(j)]=1
    return zero_df

In [ ]:
# create_onehot 함수 활용 train data >> onehot encoding
train_onehot = create_onehot(df_train, mfl_col)
train_onehot.shape

In [11]:
train_onehot.to_pickle('pickle/train_onehot.pkl')

In [35]:
# test data 준비
f_test_q = pd.read_json('data/test_cnt3_q.json' ,typ = 'frame', encoding='utf-8')
df_test_q = pd.DataFrame(f_test_q)
df_test_q.head(3)

,tags,id,plylst_title,songs,like_cnt,updt_date
0,"[알앤비, 몽환]",5931,당신의 감성을 건들일 CHILL하면서 감각적인 비트들 PBR&B,"[126214, 142857, 574222, 216846, 310802, 11138...",70,2016-12-06 14:12:37.000
1,[],142923,슬플 때 들으면 더 슬픈 뉴에이지,[],9,2013-06-01 21:14:06.000
2,"[연주음악, 주말]",23236,주말의 여유를 산뜻한 연주음악과 함께,"[330397, 533663, 497256, 436847, 136567]",13,2019-05-08 13:22:45.000


In [36]:
df_test_q.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9352 entries, 0 to 9351
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tags          9352 non-null   object
 1   id            9352 non-null   int64 
 2   plylst_title  9352 non-null   object
 3   songs         9352 non-null   object
 4   like_cnt      9352 non-null   int64 
 5   updt_date     9352 non-null   object
dtypes: int64(2), object(4)
memory usage: 438.5+ KB


In [37]:
# create_onehot 함수 활용 test data >> onehot encoding
test_q_onehot = create_onehot(df_test_q, mfl_col)
test_q_onehot.shape

100%|██████████| 9352/9352 [02:53<00:00, 53.95it/s]


(9352, 91697)

In [39]:
test_q_onehot.to_pickle('pickle/test_q_onehot.pkl')

--------------

## Arena data 원핫인코딩 (평가용)

In [10]:
# arena data 준비
f_arena_q = pd.read_json('../kakao_arena_melon/arena_data/questions/val.json' ,typ = 'frame', encoding='utf-8')
df_arena_q = pd.DataFrame(f_arena_q)
df_arena_q.head(3)

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[힐링],112148,평온한 하루를 만들며 듣는 편안한 음악,"[201175, 72649, 136416, 128673, 69515, 330081,...",6,2015-10-21 09:35:56.000
1,[],121792,상쾌한 주말을 위한 밝은느낌의 뉴에이지,"[389367, 632305, 201036, 617935, 688453, 66422...",7,2015-09-13 12:24:53.000
2,[],30753,신나는 휴가철 !!!!,"[451948, 227121, 324428, 75469, 519168, 472311...",1,2017-08-03 09:31:57.000


In [11]:
df_arena_q.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23015 entries, 0 to 23014
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tags          23015 non-null  object
 1   id            23015 non-null  int64 
 2   plylst_title  23015 non-null  object
 3   songs         23015 non-null  object
 4   like_cnt      23015 non-null  int64 
 5   updt_date     23015 non-null  object
dtypes: int64(2), object(4)
memory usage: 1.1+ MB


In [12]:
# create_onehot 함수 활용 test data >> onehot encoding
arena_q_onehot = create_onehot(df_arena_q, mfl_col)
arena_q_onehot.shape

100%|██████████| 23015/23015 [08:02<00:00, 47.70it/s]


(23015, 91697)

In [13]:
arena_q_onehot.to_pickle('pickle/arena_q_onehot.pkl')